# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 15 2022 2:24PM,253211,10,Yusen-8906636,Yusen – 3D Matrix,Released
1,Dec 15 2022 2:24PM,253211,10,Yusen-8906640,Yusen – 3D Matrix,Released
2,Dec 15 2022 2:24PM,253211,10,Yusen-8906642,Yusen – 3D Matrix,Released
3,Dec 15 2022 2:24PM,253211,10,Yusen-8906643,Yusen – 3D Matrix,Released
4,Dec 15 2022 2:24PM,253211,10,Yusen-8906644,Yusen – 3D Matrix,Released
5,Dec 15 2022 2:22PM,253210,10,0086251100,ISDIN Corporation,Released
6,Dec 15 2022 2:22PM,253210,10,0086251097,ISDIN Corporation,Released
7,Dec 15 2022 2:22PM,253210,10,0086251102,ISDIN Corporation,Released
8,Dec 15 2022 2:22PM,253210,10,0086251151,ISDIN Corporation,Released
9,Dec 15 2022 2:22PM,253210,10,0086251152,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,253207,Released,1
27,253208,Released,1
28,253209,Released,2
29,253210,Released,7
30,253211,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253207,NaN,1.0
253208,NaN,1.0
253209,NaN,2.0
253210,NaN,7.0
253211,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253139,0.0,1.0
253149,22.0,9.0
253163,2.0,21.0
253179,0.0,1.0
253181,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253139,0,1
253149,22,9
253163,2,21
253179,0,1
253181,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253139,0,1
1,253149,22,9
2,253163,2,21
3,253179,0,1
4,253181,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253139,,1
1,253149,22,9
2,253163,2,21
3,253179,,1
4,253181,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix
5,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation
12,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation
14,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC"
15,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc"
16,Dec 15 2022 1:25PM,253206,10,"Methapharm, Inc."
22,Dec 15 2022 1:22PM,253205,10,ISDIN Corporation
38,Dec 15 2022 1:15PM,253204,10,Emerginnova
39,Dec 15 2022 1:13PM,253203,10,ISDIN Corporation
43,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,,5
1,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,,7
2,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,,2
3,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",,1
4,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",,1
5,Dec 15 2022 1:25PM,253206,10,"Methapharm, Inc.",,6
6,Dec 15 2022 1:22PM,253205,10,ISDIN Corporation,,16
7,Dec 15 2022 1:15PM,253204,10,Emerginnova,,1
8,Dec 15 2022 1:13PM,253203,10,ISDIN Corporation,,4
9,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5,
1,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7,
2,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,2,
3,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",1,
4,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",1,
5,Dec 15 2022 1:25PM,253206,10,"Methapharm, Inc.",6,
6,Dec 15 2022 1:22PM,253205,10,ISDIN Corporation,16,
7,Dec 15 2022 1:15PM,253204,10,Emerginnova,1,
8,Dec 15 2022 1:13PM,253203,10,ISDIN Corporation,4,
9,Dec 15 2022 1:07PM,253201,10,"Senseonics, Incorporated",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5,
1,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7,
2,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,2,
3,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",1,
4,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5.0,NaN
1,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7.0,NaN
2,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,2.0,NaN
3,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",1.0,NaN
4,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 15 2022 2:24PM,253211,10,Yusen – 3D Matrix,5.0,0.0
1,Dec 15 2022 2:22PM,253210,10,ISDIN Corporation,7.0,0.0
2,Dec 15 2022 2:16PM,253209,10,ISDIN Corporation,2.0,0.0
3,Dec 15 2022 2:10PM,253208,12,"SD Head USA, LLC",1.0,0.0
4,Dec 15 2022 1:47PM,253207,19,"Avet Pharmaceuticals Labs, Inc",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4051146,96.0,14.0
12,759607,1.0,2.0
16,506360,2.0,0.0
19,759543,11.0,22.0
20,253139,1.0,0.0
21,506391,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4051146,96.0,14.0
1,12,759607,1.0,2.0
2,16,506360,2.0,0.0
3,19,759543,11.0,22.0
4,20,253139,1.0,0.0
5,21,506391,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,96.0,14.0
1,12,1.0,2.0
2,16,2.0,0.0
3,19,11.0,22.0
4,20,1.0,0.0
5,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,12,Released,1.0
2,16,Released,2.0
3,19,Released,11.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,14.0,2.0,0.0,22.0,0.0,0.0
Released,96.0,1.0,2.0,11.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,14.0,2.0,0.0,22.0,0.0,0.0
1,Released,96.0,1.0,2.0,11.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,14.0,2.0,0.0,22.0,0.0,0.0
1,Released,96.0,1.0,2.0,11.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()